In [2]:
import ee

ee.Initialize()

In [3]:
def split(features: ee.FeatureCollection) -> tuple[ee.FeatureCollection, ee.FeatureCollection]:
    train = features.filter(ee.Filter.eq('type', 1))
    test = features.filter(ee.Filter.eq('type', 2))
    return train, test

In [4]:
features = ee.FeatureCollection("projects/ee-nwrc-geomatics/assets/cnwi/sampled_features_122")

In [5]:
X,Y = split(features)

In [6]:
# load the predictor variables
with open("./channel_log.txt", 'r') as f:
    channel_log = f.read().splitlines() 
print(channel_log[0:2])

['HH', 'HV']


In [7]:
from cnwi.cnwilib.random_forest import SmileRandomForest

model = SmileRandomForest()
model.fit(
    features=X,
    classProperty='class_name',
    inputProperties=channel_log, 
)


In [8]:
predict = model.predict(Y)

In [9]:
from cnwi.cnwilib.assessment import AssessmentTable
assessment = AssessmentTable(
    predictions=predict,
    actual='class_name',
    class_order=features.aggregate_array('class_name').distinct(),
)

cfm = assessment.create_metrics()


In [10]:
# task = ee.batch.Export.table.toDrive(
#     collection=cfm,
#     description='cnwi_cfm',
#     fileFormat='GeoJSON'
# )
# task.start()




In [11]:
# export the model
model_task = model.save_model(asset_name='projects/ee-nwrc-geomatics/assets/cnwi/models/cnwi_model_122')
model_task.start()